In [7]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 32

max_iters =10000
learning_rate=3e-4
eval_iters=250
dropout = 0.2
n_embd = 32

cuda


In [2]:
with open('wizard_of_oz.txt','r',encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string= { i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s ]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n= int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x= torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y

x,y=get_batch('train')
print(x)
print(y)

 



tensor([[ 1, 57, 68, 68, 71,  1, 76, 62],
        [ 1, 76, 68, 68, 57, 58, 67,  1],
        [58, 57,  1, 73, 61, 58,  0, 72],
        [76, 61, 58, 67,  1, 61, 58,  1]], device='cuda:0')
tensor([[57, 68, 68, 71,  1, 76, 62, 73],
        [76, 68, 68, 57, 58, 67,  1, 72],
        [57,  1, 73, 61, 58,  0, 72, 68],
        [61, 58, 67,  1, 61, 58,  1, 72]], device='cuda:0')


In [5]:
#reduce compute when model is used
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y=get_batch(split)
            logits ,loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
        

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table=nn.Embedding(block_size,n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)
        
    def forward(self,index,targets=None):
        B,T = index.shape
        tok_emb= self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device))
        logits = self.lm_head(tok_emb)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            targets = targets.to(logits.device)
            loss = F.cross_entropy(logits,targets)
        
        return logits ,loss
        
    def generate(self,index,max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss= self.forward(index)
            logits = logits[:, -1,:]
            probs = F.softmax(logits,dim=-1)
            index_next = torch.multinomial(probs,num_samples=1)
            index = torch.cat((index,index_next),dim = 1)
        return index
model = BigramLanguageModel()
m=model.to(device) 
logits,loss= m(x,y)

context = torch.zeros((1,1), dtype =torch.long,device=device)
generated_chars = decode(m.generate(context , max_new_tokens=100)[0].tolist())
print(generated_chars)



AttributeError: 'BigramLanguageModel' object has no attribute 'position_embedding_table'

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [ ]:
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses=estimate_loss()
        print(f' Step:{iter} , loss = {losses}')
        
    xb,yb= get_batch('train')
    logits , loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype =torch.long,device=device)
generated_chars = decode(m.generate(context , max_new_tokens=500)[0].tolist())
print(generated_chars)

